<a href="https://colab.research.google.com/github/GreatLearningAIML1/gurgaon-feb-batch-abhishek02git/blob/master/Project_1_CV_with_CNN_Plant_Seedling_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import tensorflow as tf
# import os
# tf_ver = tf.__version__
# print (tf_ver)
# if not(tf_ver[0]=='2'):
#   !pip install tensorflow==2.0.0b1
#   #!pip install -q tf-nightly-2.0-preview
# else:
#   print (tf_ver, "Already Installed")
  

# #%tensorboard --logdir {logs_dir}

In [2]:
# Import Libraries

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers


from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

from tensorflow.python.keras.applications import ResNet50
from tensorflow.keras.applications.xception import Xception



from keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.models import load_model


import os
import glob
import itertools

import cv2
from skimage import io

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

logs_dir = "drive//My Drive//seedling//logs"
#os.makedirs(logs_dir, exist_ok=True)

img_width = 251
img_height = 251
channels = 3
seed = 2019

Using TensorFlow backend.


In [3]:
# Copy Files from my Google Drive and Unzip
if len(glob.glob('train'))==0:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=False)
  
  #! wget 'http://h0.ai/static/ml/seeding/train.zip'
  #! cp 'train.zip' 'drive//My Drive//seedling'
  !cp "drive//My Drive//seedling//train.zip" "./"
  ! unzip train.zip
  #! wget 'http://h0.ai/static/ml/seeding/test.zip'
  #! cp 'test.zip' 'drive//My Drive//seedling'
  !cp "drive//My Drive//seedling//test.zip" "./"
  ! unzip test.zip
!ls train

 Black-grass	    'Common wheat'	'Scentless Mayweed'
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
# Help Functions


def train_test_split_df (df,train_size):
   
  rnd = np.random.random(len(df))
  train_idx = rnd<train_size
  test_idx = rnd>=train_size
  df_train = df[train_idx].reset_index(drop=True)
  df_test = df[test_idx].reset_index(drop=True)
    
  return df_train,df_test

def train_test_split (X,y,train_size):
   
  rnd = np.random.random(len(X))
  train_idx = rnd<train_size
  test_idx = rnd>=train_size
  X_train = X[train_idx]
  X_test = X[test_idx]
  y_train = y[train_idx]
  y_test = y[test_idx]
  
  X_train = X_train.reset_index(drop=True)
  X_test = X_test.reset_index(drop=True)
  y_train = y_train.reset_index(drop=True)
  y_test = y_test.reset_index(drop=True)
  
  return X_train,X_test,y_train,y_test

def read_img(filepath, img_width,img_height):
    img = cv2.imread(filepath)
    #plt.imshow(img)
    img = cv2.resize(img,(img_width,img_height))
    
    return img

In [5]:
# Define class from Folder Names
train_classes = os.listdir('train')
train = []
for idx, class_name in enumerate(train_classes):
  path = 'train/' + class_name
  for file in os.listdir(path):
    train.append(['train/{}/{}'.format(class_name,file),idx,class_name])
              
df = pd.DataFrame(train,columns =['FileName','ClassID','ClassName'] )

train_df, test_df = train_test_split_df(df,train_size=0.8)
X_train = train_df['FileName']
y_train = train_df['ClassID']
X_test = test_df['FileName']
y_test = test_df['ClassID']
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

print ("No of Samples in Trainn are {}" .format(X_train.shape))
print ("No of Samples in testing are {}" .format (X_test.shape))
print ("Classes: ")

x1_df = pd.DataFrame({'Total_Count':df.groupby(by='ClassName').count()['FileName'],'Training':train_df.groupby(by='ClassName').count()['FileName'],
                      'Testing':test_df.groupby(by='ClassName').count()['FileName']})
x1_df

No of Samples in Trainn are (3856,)
No of Samples in testing are (894,)
Classes: 


,Total_Count,Training,Testing
ClassName,,,
Black-grass,263,210,53
Charlock,390,319,71
Cleavers,287,245,42
Common Chickweed,611,490,121
Common wheat,221,173,48
Fat Hen,475,390,85
Loose Silky-bent,654,526,128
Maize,221,176,45
Scentless Mayweed,516,433,83


In [6]:
X_test_img = np.ndarray(shape=(len(X_test), img_height, img_width, channels),
                     dtype=np.float32)
print ("\n Loading Test Images ", end =" ")

for idx, files in enumerate(range(len(X_test))):
  X_test_img[idx] = read_img(X_test[idx], img_width,img_height)
  
  if np.mod(idx,50)==0: print ("#",end='')

X_train_img = np.ndarray(shape=(len(X_train), img_height, img_width, channels),
                     dtype=np.float32)

y_train_img =  np.ndarray(shape=(len(X_train),12),dtype=np.float32)

print ("\n Loading Train Images ", end =" ")

for idx, files in enumerate(range(len(X_train))):
    X_train_img[idx] = read_img(X_train[idx], img_width,img_height)
    y_train_img[idx] = y_train[idx]
    
    if np.mod(idx,50)==0: print ("#",end='')


 Loading Test Images  ##################
 Loading Train Images  ##############################################################################

In [7]:
train_datagen = ImageDataGenerator(
      featurewise_std_normalization=True,
      samplewise_std_normalization = False,
      rotation_range=180,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      vertical_flip=False,
      fill_mode='nearest',
      zoom_range=0.2)

train_datagen.fit(X_train_img)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:348: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [8]:
TRAIN = True
Transfer_Model = True

# If TRAIN = True then creat a new graph else load saved model
if TRAIN:
  model = models.Sequential()
  model.add(BatchNormalization(input_shape=(img_width,img_height,3)))

# If Transfer_Model = True then use a pre trained model else create convolution model
  if Transfer_Model:
    #transfer = VGG16(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
    
    transfer = ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet',input_shape=(img_width,img_height,3))
    #transfer = Xception(include_top = False, pooling = 'avg', weights = 'imagenet',input_shape=(img_width,img_height,3))
    
    
# Make only last 3 layers trainable
    for layer in transfer.layers[:-3]:
         layer.trainable = False
      
# Check the layers
    for layer in transfer.layers:
      print(layer, layer.trainable)
      
# Add pretrained model to graph
    model.add(transfer)
  else:
    model.add(Conv2D(60,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(40,(3,3),activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(20,(3,3),activation='relu'))
    model.add(Flatten())

# Create a dense layer for output
  model.add(Dropout(0.2))
  model.add(BatchNormalization())
  model.add(layers.Dense(12, activation='softmax'))
    
else:
  model = load_model("drive//My Drive//seedling/model.h5")
      

# Show a summary of the model. Check the number of trainable parameters
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
tb = tf.keras.callbacks.TensorBoard(logs_dir)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7ff08d2cfeb8> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7ff08d2db668> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff08ca01d68> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7ff08ca1abe0> False
<tensorflow.python.keras.layers.core.Activation object at 0x7ff08c9b55f8> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7ff08c9c5358> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7ff08c9611d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff08c95cfd0> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7ff08c8f9668> False
<tensorflow.python.keras.layers.core.Activation object at 0x7ff08c9041d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff08c904278> False
<tensorflow.python.keras.layers.nor

W0816 09:11:56.842189 140672799401856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 251, 251, 3)       12        
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 12)                24588     
Total params: 23,620,504
Trainable params: 28,690
Non-trainable params: 23,591,814
_________________________________________________________________


In [0]:
# Assign class weight as data is imbalanced
class_weight = {0:3,1:1,2:1,3:1,4:3,5:1,6:2,7:2,8:1,9:2,10:1.5,11:1.5}

# Run short cycle of iteriations so that model is frequenty saved
for i in range (100):
  model.fit_generator(train_datagen.flow(X_train_img, y_train_img,batch_size=32),
                      epochs=5, 
                      steps_per_epoch= X_train_img.shape[0]/32,
                      validation_data=(X_test_img, y_test),
                      class_weight = class_weight,
                      callbacks=[])

  model.save('model.h5')
  print ("model saved")
  !cp './/model.h5'  "drive//My Drive//seedling"
  #validation_data=(X_test_img, y_test), epochs=epochs, batch_size=32,callbacks=[es]

Epoch 1/5
121/120 [==============================] - 104s 862ms/step - loss: 2.7439 - acc: 0.4448 - val_loss: 2.5781 - val_acc: 0.3065
Epoch 2/5
121/120 [==============================] - 101s 832ms/step - loss: 1.5578 - acc: 0.6522 - val_loss: 2.6262 - val_acc: 0.3691
Epoch 3/5
121/120 [==============================] - 101s 831ms/step - loss: 1.3095 - acc: 0.7025 - val_loss: 2.4695 - val_acc: 0.4385
Epoch 4/5
121/120 [==============================] - 101s 838ms/step - loss: 1.1841 - acc: 0.7339 - val_loss: 2.2742 - val_acc: 0.4541
Epoch 5/5
121/120 [==============================] - 102s 840ms/step - loss: 1.0981 - acc: 0.7477 - val_loss: 2.1242 - val_acc: 0.4799
model saved
Epoch 1/5
121/120 [==============================] - 104s 856ms/step - loss: 0.9918 - acc: 0.7707 - val_loss: 2.9784 - val_acc: 0.4262
Epoch 2/5
121/120 [==============================] - 102s 844ms/step - loss: 0.9769 - acc: 0.7809 - val_loss: 1.8538 - val_acc: 0.5570
Epoch 3/5
121/120 [========================

In [0]:
model.predict(y_test_img[0].reshape(1,251,251,3))

In [0]:
df = pd.DataFrame(y_train_img)

In [0]:
df.sum()/len(df)*100

In [0]:
sns.distplot(df.sum()/len(df)*100)